In [1]:
# from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey, Table
from sqlalchemy.orm import relationship
# from sqlalchemy.orm import sessionmaker
# from sqlalchemy.ext.declarative import declarative_base
import sys
from sqlapp import SqlApp
import functools

import pprint

In [2]:
s = SqlApp('postgresql://postgres@localhost:5532/sqlapp')

In [3]:
s.connect('postgresql://postgres@localhost:5532/sqlapp')

In [3]:
#Class Method Destructuring
import functools
populate, create, Base = map(functools.partial(getattr, s), ['populate', 'create', 'Base'])

In [6]:
# /!\ DROP MODEL IN DB /!\
s.drop_models() 

TypeError: drop_models() takes exactly 2 arguments (1 given)

## Many To One Relationship

In [7]:
"""
A basic Many to One Relationship
"""

class PlayerModel(Base):
    """Data model for players."""
    __tablename__ = "players"
 
    id = Column(Integer,
                primary_key=True,
                nullable=False)
    team_id = Column(Integer,
                     ForeignKey('teams.id'),
                     nullable=False)
    name = Column(String(100),
                  nullable=False)
    position = Column(String(100),
                      nullable=False)

    # Relationships
    team = relationship("TeamModel", backref="players")
    
    def __repr__(self):
        return '<Person model {}>'.format(self.id)


class TeamModel(Base):
    """Data model for people."""
    __tablename__ = "teams"

    id = Column(Integer,
                primary_key=True,
                nullable=False,
    )
    name = Column(String(100),
                  nullable=False)
    city = Column(String(100),
                  nullable=False)

    def __repr__(self):
        return '<Team model {}>'.format(self.id)

In [8]:
# PlayerModel.__table__.drop(engine)
# TeamModel.__table__.drop(engine)

# Many 2 Many Relationship

In [4]:


association_table = Table('association', Base.metadata,
    Column('left_id', Integer, ForeignKey('left.id')),
    Column('right_id', String, ForeignKey('right.id'))
)


class Parent(Base):
    __tablename__ = 'left'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    children = relationship(
        "Child",
        secondary=association_table)


class Child(Base):
    __tablename__ = 'right'
    id = Column(String, primary_key=True, )

In [5]:
# /!\ WRITE MODEL IN DB /!\
# Base.metadata.create_all(engine)
s.create_models(Base)

2020-02-09 01:52:45,555 INFO sqlalchemy.engine.base.Engine select version()
2020-02-09 01:52:45,557 INFO sqlalchemy.engine.base.Engine {}
2020-02-09 01:52:45,565 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-02-09 01:52:45,566 INFO sqlalchemy.engine.base.Engine {}
2020-02-09 01:52:45,576 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-02-09 01:52:45,578 INFO sqlalchemy.engine.base.Engine {}
2020-02-09 01:52:45,584 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-02-09 01:52:45,586 INFO sqlalchemy.engine.base.Engine {}
2020-02-09 01:52:45,601 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-02-09 01:52:45,603 INFO sqlalchemy.engine.base.Engine {}
2020-02-09 01:52:45,614 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

In [12]:
TEAMS = [
    { 'name': 'PSG', 'city': 'Paris', 'id': 1},
    { 'name': 'OM', 'city': 'Marseille', 'id': 2},
    { 'name': 'OL', 'city': 'Lyon', 'id': 3},
    { 'name': 'LOSC', 'city': 'Lille', 'id': 4},
]
populate(TeamModel, TEAMS)

2020-02-09 01:41:14,848 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-09 01:41:14,850 INFO sqlalchemy.engine.base.Engine INSERT INTO teams (id, name, city) VALUES (%(id)s, %(name)s, %(city)s)
2020-02-09 01:41:14,852 INFO sqlalchemy.engine.base.Engine ({'city': 'Paris', 'name': 'PSG', 'id': 1}, {'city': 'Marseille', 'name': 'OM', 'id': 2}, {'city': 'Lyon', 'name': 'OL', 'id': 3}, {'city': 'Lille', 'name': 'LOSC', 'id': 4})
2020-02-09 01:41:14,860 INFO sqlalchemy.engine.base.Engine ROLLBACK


ProgrammingError: (psycopg2.ProgrammingError) relation "teams" does not exist
LINE 1: INSERT INTO teams (id, name, city) VALUES (1, 'PSG', 'Paris'...
                    ^
 [SQL: 'INSERT INTO teams (id, name, city) VALUES (%(id)s, %(name)s, %(city)s)'] [parameters: ({'city': 'Paris', 'name': 'PSG', 'id': 1}, {'city': 'Marseille', 'name': 'OM', 'id': 2}, {'city': 'Lyon', 'name': 'OL', 'id': 3}, {'city': 'Lille', 'name': 'LOSC', 'id': 4})]

In [47]:
PLAYERS = [
    {'name':'Mbappe', 'position': '9', 'team_id': '1'},
    {'name':'Neymar', 'position': '7', 'team_id': '1'},
    {'name':'Diawara', 'position': '8', 'team_id': '3'},
    {'name':'Benedeto', 'position': '9', 'team_id': '2'},
    {'name':'Payet', 'position': '7', 'team_id': '2'},
    {'name':'Pepe', 'position': '10', 'team_id': '4'},
]
populate(PlayerModel, PLAYERS)

2020-02-05 00:13:14,727 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-05 00:13:14,731 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-05 00:13:14,733 INFO sqlalchemy.engine.base.Engine {'position': '9', 'team_id': '1', 'name': 'Mbappe'}
2020-02-05 00:13:14,747 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-05 00:13:14,748 INFO sqlalchemy.engine.base.Engine {'position': '7', 'team_id': '1', 'name': 'Neymar'}
2020-02-05 00:13:14,757 INFO sqlalchemy.engine.base.Engine INSERT INTO players (team_id, name, position) VALUES (%(team_id)s, %(name)s, %(position)s) RETURNING players.id
2020-02-05 00:13:14,762 INFO sqlalchemy.engine.base.Engine {'position': '8', 'team_id': '3', 'name': 'Diawara'}
2020-02-05 00:13:14,771 INFO sqlalchemy.engine.base.Engine INSERT INTO player

In [6]:
populate(Parent, [{'id':1, 'name': 'x'}, {'id':2, 'name': 'y'}])
populate(Child, [
    {'id': 'FR',}, {'id': 'ES'}, {'id': 'IT'},{'id': 'EN'}
])

2020-02-09 01:52:51,916 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-09 01:52:51,919 INFO sqlalchemy.engine.base.Engine INSERT INTO "left" (id, name) VALUES (%(id)s, %(name)s)
2020-02-09 01:52:51,921 INFO sqlalchemy.engine.base.Engine ({'id': 1, 'name': 'x'}, {'id': 2, 'name': 'y'})
2020-02-09 01:52:51,931 INFO sqlalchemy.engine.base.Engine COMMIT
2020-02-09 01:52:51,937 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-09 01:52:51,940 INFO sqlalchemy.engine.base.Engine INSERT INTO "right" (id) VALUES (%(id)s)
2020-02-09 01:52:51,941 INFO sqlalchemy.engine.base.Engine ({'id': 'FR'}, {'id': 'ES'}, {'id': 'IT'}, {'id': 'EN'})
2020-02-09 01:52:51,964 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
p = s.session.query(Parent).filter_by(id=1).first()

2020-02-09 01:52:56,620 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-02-09 01:52:56,623 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id, "left".name AS left_name 
FROM "left" 
WHERE "left".id = %(id_1)s 
 LIMIT %(param_1)s
2020-02-09 01:52:56,625 INFO sqlalchemy.engine.base.Engine {'id_1': 1, 'param_1': 1}


In [52]:
p.children

2020-02-05 15:27:05,329 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right", association 
WHERE %(param_1)s = association.left_id AND "right".id = association.right_id
2020-02-05 15:27:05,331 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[]

In [53]:
p.children.append(s.session.query(Child).filter_by(id='ES').first())

2020-02-05 15:27:08,128 INFO sqlalchemy.engine.base.Engine SELECT "right".id AS right_id 
FROM "right" 
WHERE "right".id = %(id_1)s 
 LIMIT %(param_1)s
2020-02-05 15:27:08,129 INFO sqlalchemy.engine.base.Engine {'id_1': 'ES', 'param_1': 1}


In [54]:
create(p)

2020-02-05 15:27:09,276 INFO sqlalchemy.engine.base.Engine INSERT INTO association (left_id, right_id) VALUES (%(left_id)s, %(right_id)s)
2020-02-05 15:27:09,278 INFO sqlalchemy.engine.base.Engine {'left_id': 1, 'right_id': u'ES'}
2020-02-05 15:27:09,285 INFO sqlalchemy.engine.base.Engine COMMIT


In [57]:
p.children[0].id

u'ES'